In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Dropdown, fixed
from IPython.display import display
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

In [2]:
# Path to go up one level from the current folder to the parent folder
parent_path = os.path.join(os.getcwd(), '..')

# Path to the target data folder
data_folder_path = os.path.join(parent_path, 'Data')

In [3]:
menu_df = pd.read_csv(os.path.join(data_folder_path, 'menu_analysis.csv'))
customers_df = pd.read_csv(os.path.join(data_folder_path, 'customers.csv'))
stores_df = pd.read_csv(os.path.join(data_folder_path, 'stores.csv'))
structure_df = pd.read_csv(os.path.join(data_folder_path, 'structure.csv'))
size_df = pd.read_csv(os.path.join(data_folder_path, 'order_size.csv'))
transactions_df = pd.read_csv(
        os.path.join(data_folder_path, "transactions.csv"),
        parse_dates=['Transaction_Date']  # Modify as needed if format issues arise
    )

In [9]:
menu_df.head()

,Group,Item_Number,Ingredient,Price,Cost,Points Earn,Points Redeem
0,Primary,Item_1,Chicken,6.5,2.0,65.0,400.0
1,Primary,Item_2,Beef,8.0,2.5,80.0,500.0
2,Primary,Item_3,Shrimp,7.5,3.0,75.0,600.0
3,Primary,Item_4,Tofu,5.5,1.0,55.0,200.0
4,Primary,Item_5,Chickpeas,4.5,0.5,45.0,100.0


### Modification 1

In [4]:
# Assuming transactions_df has columns: 'Transaction_Date', 'Store_Number', 'Price'
transactions_df['Transaction_Date'] = pd.to_datetime(transactions_df['Transaction_Date'])

### Modification 2

In [10]:
transactions_df = pd.merge(transactions_df, menu_df[['Item_Number', 'Price']], on='Item_Number')

### Modification 3

In [11]:
# Merge transactions with menu data
merged_df = pd.merge(transactions_df, menu_df[['Item_Number', 'Group']], on='Item_Number')

In [12]:
merged_df.head()

,Order_Number,Transaction_Date,Transaction_Time,Store_Number,Customer_ID,Item_Number,Price,Group
0,142292,2024-01-14,00:43:25,STR_7,CUST3706,Item_4,5.5,Primary
1,134538,2023-07-22,03:11:53,STR_10,CUST2840,Item_4,5.5,Primary
2,105529,2024-06-20,08:51:22,STR_8,CUST1036,Item_4,5.5,Primary
3,34280,2024-05-06,17:40:34,STR_5,CUST0994,Item_4,5.5,Primary
4,118343,2023-11-07,00:32:02,STR_5,CUST2229,Item_4,5.5,Primary


### Modification 4

In [13]:
# Calculate revenue by group, store, and date
revenue_by_group = merged_df.groupby(['Transaction_Date', 'Store_Number', 'Group'])['Price'].sum().reset_index()

In [14]:
revenue_by_group.head()

,Transaction_Date,Store_Number,Group,Price
0,2023-06-21,STR_1,Beverage,11.0
1,2023-06-21,STR_1,Primary,320.5
2,2023-06-21,STR_10,Beverage,8.5
3,2023-06-21,STR_10,Primary,335.5
4,2023-06-21,STR_2,Beverage,15.5


# Modification 5

In [16]:
# Get unique stores and groups
stores = revenue_by_group['Store_Number'].unique()
groups = revenue_by_group['Group'].unique()

In [18]:
print(stores[0:5])
print(groups[0:5])

['STR_1' 'STR_10' 'STR_2' 'STR_3' 'STR_4']
['Beverage' 'Primary']


# 7. Revenue Distribution by Menu Group

In [20]:
# Create the plot
fig = go.Figure()

for store in stores:
    store_data = revenue_by_group[revenue_by_group['Store_Number'] == store]
    
    fig.add_trace(go.Bar(
        x=store_data['Transaction_Date'],
        y=store_data[store_data['Group'] == groups[0]]['Price'],
        name=f'{groups[0]} - Store {store}',
        offsetgroup=store,
        customdata=store_data[store_data['Group'] == groups[0]][['Store_Number', 'Group']],
        hovertemplate='<b>Store %{customdata[0]}</b><br>Date: %{x|%Y-%m-%d}<br>Group: %{customdata[1]}<br>Revenue: $%{y:.2f}<extra></extra>'
    ))
    
    for group in groups[1:]:
        fig.add_trace(go.Bar(
            x=store_data['Transaction_Date'],
            y=store_data[store_data['Group'] == group]['Price'],
            name=f'{group} - Store {store}',
            offsetgroup=store,
            base=store_data[store_data['Group'].isin(groups[:groups.tolist().index(group)])].groupby('Transaction_Date')['Price'].sum(),
            customdata=store_data[store_data['Group'] == group][['Store_Number', 'Group']],
            hovertemplate='<b>Store %{customdata[0]}</b><br>Date: %{x|%Y-%m-%d}<br>Group: %{customdata[1]}<br>Revenue: $%{y:.2f}<extra></extra>'
        ))

fig.update_layout(
    title='Revenue Distribution by Menu Group',
    xaxis_title='Transaction_Date',
    yaxis_title='Revenue ($)',
    barmode='stack',
    legend_title='Menu Group - Store',
    hovermode='closest'
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

fig.show()